# Archived Chat Processor

### Import DF

In [99]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

number_of_csv = 14

df = pd.read_csv(f"{number_of_csv}.csv")

df = df.iloc[::-1]

question = {str(df.iloc[0]['author']): df.iloc[0]['content']}

answers = []
for i in range(1, len(df)):
    answers.append({str(df.iloc[i]['author']): df.iloc[i]['content']})

In [100]:
data = [
    question,
    answers
]

In [101]:
import json

file_path = f"processed{number_of_csv}.txt"
json_data = json.dumps(data, indent=2)

with open(file_path, 'w') as file:
    file.write(json_data)

print(f'Data has been exported to {file_path}')

Data has been exported to processed14.txt


### Assistant

In [40]:
import openai
client = OpenAI(
    api_key="",
)
print(openai.VERSION)

1.2.2


In [102]:
assistant = client.beta.assistants.create(
    name="Discord Thread Inferer",
    instructions="""
        You have txt file named processed14.txt.
        It is thread datas from a Discord QA Forum Channel.
        
        First object is the author and the question
        Other array of objects are actually answers to the question
        
        Infer, evaulate and create Full Question: Detailed Answer Pair
    """,
    tools=[{"type": "retrieval"},{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

In [103]:
file = client.files.create(
  file=open(file_path, "rb"),
  purpose="assistants"
)

In [105]:
assistant_file = client.beta.assistants.files.create(
  assistant_id=assistant.id, 
  file_id=file.id
)

In [106]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create Full Question: Detailed Answer pair"
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

import time
# Add more time if cannot retrieve
time.sleep(45)

runned = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)

In [96]:
text = messages.data[0].content[0].text.value
question_index = text.find("**Full Question:**")
answer_index = text.find("**Detailed Answer:**")

full_question = text[question_index + len("**Full Question:**"):answer_index].strip()
detailed_answer = text[answer_index + len("**Detailed Answer:**"):].strip()

result_json = {
    "FullQuestion": full_question,
    "DetailedAnswer": detailed_answer
}

print(json.dumps(result_json, indent=2))

{
  "FullQuestion": "ion: Detailed Answer pai",
  "DetailedAnswer": "n: Detailed Answer pair"
}
